# Base de datos Riak

In [1]:
import riak
from pprintpp import pprint as pp
import json
import pandas as pd

import uuid
import hashlib

## Conexion a RIAK

In [2]:
# connect to database
cliente = riak.RiakClient()
cliente.ping()

True

## Funciones utiles para el desarrollo de la práctica

In [3]:
# ELimina datos de un Bucket
def drop_keys(bucket):
    for keys in bucket.stream_keys():
        # print(keys)
        for keys in bucket.stream_keys():
            for key in keys:
                bucket.delete(key)

In [4]:
# Devuelve el hash de un string
def hash_string(s):
    s_utf8 = s.encode('utf-8')
    return hashlib.md5(s_utf8).hexdigest()

In [5]:
# Convertimos el objeto JSON en un objeto pandas 
def json_to_pandas(jsonObj):
    return pd.read_json(json.dumps(jsonObj))

## Lectura de la informacion en Pandas

In [6]:
df_dificultades = pd.read_csv("./data/dificultades.csv",encoding='utf-8');
df_encadenamientos = pd.read_csv("./data/tipos_encadenamiento.csv", encoding='utf-8');
df_escaladores = pd.read_csv("./data/escaladores_lite_2017.csv", encoding='utf-8');
df_ascensos = pd.read_csv("./data/ascensos_lite_2017.csv", encoding='utf-8');

In [7]:
# 'merge' del tipo de encadenamiento en el data frame de ascensos
df_ascensos = pd.merge(df_ascensos, df_encadenamientos, on = ['id_tipo_encadenamiento'], how = 'inner')
df_ascensos = df_ascensos[['id_escalador','id_dificultad', 'tipo_encadenamiento', 'nombre_via', 'risco', 'sector', 'pais']];


## Desnormalización del data frame de ascensos

Voy a incluir la información del usuario en cada registro de ascenso, para ir creando los buckets que van interesando según las preguntas a resolver


In [8]:
df_ascensos_desnormalizado = pd.merge(df_ascensos, df_escaladores, on = ['id_escalador'], how = 'inner');
df_ascensos_desnormalizado = df_ascensos_desnormalizado.rename(
    columns={
        'pais_x' : 'pais_risco',
        'ciudad':'ciudad_escalador',
        'pais_y':'pais_escalador'});

df_ascensos_desnormalizado.head()

,id_escalador,id_dificultad,tipo_encadenamiento,nombre_via,risco,sector,pais_risco,nombre,sexo,fecha_nacimiento,ciudad_escalador,pais_escalador,comienzo
0,19961,33,Onsight,MEGA DIRETTORE,ARCO,NAGO,ITA,Jos Dehaes,Hombre,1973-05-08,Leuven,BEL,2008
1,19961,29,Onsight,SUPER MAZOUT,BOMAL,LE CROIX,BEL,Jos Dehaes,Hombre,1973-05-08,Leuven,BEL,2008
2,19961,33,Onsight,PICCOLA CHIMERA,ARCO,BELVEDERE,ITA,Jos Dehaes,Hombre,1973-05-08,Leuven,BEL,2008
3,19961,33,Onsight,LES MUGUETS,ROCHERS DE LA VIERGE,ARéTE DE CHIRMONT - FACE SUD,BEL,Jos Dehaes,Hombre,1973-05-08,Leuven,BEL,2008
4,19961,36,Onsight,JACQUOUILLE LA FRIPOUILLE,FREYR,AL LEGNE,BEL,Jos Dehaes,Hombre,1973-05-08,Leuven,BEL,2008


In [9]:
ascensosJsonString = df_ascensos_desnormalizado.to_json(orient = 'records')
ascensosJson = json.loads(ascensosJsonString)

## Estructura RIAK en función de las preguntas

### 1.a) Los 10 escaladores (hombres) más activos.
### 1.b) Los 10 escaladoras (mujeres) más activas.

Para resolver estas preguntas me interesa tener un bucket especifico con los ascensos acumulados por escalador:

|bucket|clave|Contenido|
|-|-|-|
|ascensos_acum|id_escalador|Mapa con el nombre, sexo, pais y ascensos_acumulados|




**Creacion de buckets e insercion de datos en RIAK**

In [10]:
BUCKET_ACUM_ASCENSOS = 'acum_ascensos'
bucket_acum_ascensos = cliente.bucket_type('maps').bucket(BUCKET_ACUM_ASCENSOS)
drop_keys(bucket_acum_ascensos)

In [11]:
# Bucle de carga de datos ascensos acumulados ...
for ascensoJson in ascensosJson:
    # construcción de ascensos acumulados por persona
    acumulador_ascensos = bucket_acum_ascensos.new(str(ascensoJson["id_escalador"]))
    acumulador_ascensos.registers['nombre'].assign(ascensoJson["nombre"].encode('utf-8'))
    acumulador_ascensos.registers['sexo'].assign(ascensoJson["sexo"].encode('utf-8'))
    acumulador_ascensos.registers['pais'].assign(ascensoJson["pais_escalador"].encode('utf-8'))
    acumulador_ascensos.counters['numero_vias'].increment(1)
    acumulador_ascensos.store()

Para obtener la información de cuantos ascensos se han realizado por escalador recurriré a consultar el mapa que acabamos de crear en RIAK:

In [12]:
ascensosAcumuladosJson = []
for streamKeys in bucket_acum_ascensos.stream_keys():
    for streamKey in streamKeys:
        acumulador_ascensos = bucket_acum_ascensos.get(streamKey)
        ascensosAcumuladosJson.append({"id_escalador" : streamKey,
                                       "nombre" : acumulador_ascensos.registers['nombre'].value,
                                       "sexo" : acumulador_ascensos.registers['sexo'].value,
                                       "pais" : acumulador_ascensos.registers['pais'].value,
                                       "numero_vias" : acumulador_ascensos.counters['numero_vias'].value})
        
# Convertimos el objeto json en un objeto pandas 
df_ascensosAcumulados = json_to_pandas(ascensosAcumuladosJson)

**RESPUESTA 1.a)**

In [13]:
df_ascensosAcumuladosHombres = df_ascensosAcumulados[df_ascensosAcumulados['sexo']=='Hombre'];
df_ascensosAcumuladosHombres = df_ascensosAcumuladosHombres.sort_values(['numero_vias', 'id_escalador'],ascending=[False, False]);
df_ascensosAcumuladosHombres.head(10)

,id_escalador,nombre,numero_vias,pais,sexo
1595,50884,Christopher Leonetti,50,USA,Hombre
1785,20095,Matthias Schuster,37,DEU,Hombre
3740,62639,Artur Gryt,20,POL,Hombre
4134,9171,Laurenz Trawnicek,20,AUT,Hombre
1025,42086,Marcin Opozda,19,POL,Hombre
1343,35082,Filip Notebaert,17,BEL,Hombre
262,28498,Lukasz Harazin,17,POL,Hombre
282,20384,Clemens Kurth,17,DEU,Hombre
2501,66250,Kuba Kaminski,16,POL,Hombre
1368,26155,Valentin Sattek,16,AUT,Hombre


**RESPUESTA 1.b)**

In [14]:
df_ascensosAcumuladosMujeres = df_ascensosAcumulados[df_ascensosAcumulados['sexo']=='Mujer'];
df_ascensosAcumuladosMujeres = df_ascensosAcumuladosMujeres.sort_values(['numero_vias', 'id_escalador'],ascending=[False, False]);
df_ascensosAcumuladosMujeres.head(10)

,id_escalador,nombre,numero_vias,pais,sexo
2374,53983,Daniela Bärtschi,15,CHE,Mujer
3383,21011,Roxane DURAND,13,FRA,Mujer
3939,65502,Ksenia Targosz,11,POL,Mujer
4274,65194,Kater Karlo,11,DEU,Mujer
2311,65069,Mania Mania A.,11,POL,Mujer
2429,60087,Ivana Balic,11,HRV,Mujer
906,49569,Karina Kosiorek,11,POL,Mujer
3902,48798,Ba Lu,10,POL,Mujer
258,29821,Wilma Thurkow,10,NLD,Mujer
571,22291,Stella Mascari,10,USA,Mujer


### 2. Lista de los 10 ascensos "On sight" de la escaladora más activa en orden decreciente de dificultad


In [15]:
BUCKET_ASCENSOS = 'ascensos'
bucket_ascensos = cliente.bucket(BUCKET_ASCENSOS)
drop_keys(bucket_ascensos)


In [17]:
# Bucle de carga de datos de ascensos ...
for ascensoJson in ascensosJson:
    key = uuid.uuid1().hex
    ascenso = bucket_ascensos.new(key, ascensoJson)
    ascenso.add_index('idx_escalador_int', ascensoJson["id_escalador"])
    ascenso.store()


### 3. Dificultad media de los ascensos del escalador más activo

### 4.a) Los 10 ascensos mas dificiles

### 4.b) Los 10 ascensos mas dificiles a vista (On sight)

### 5.a) Grado medio y maximo de los ascensos en España de los 10 escaladores NO ESPAÑOLES con mas ascensos en España

### 5.b) Grado medio y maximo de los ascensos en España de los 10 escaladores ESPAÑOLES con mas ascensos en España

### 6.a) Dificultad media y maxima de los ascensos NO "Top Rope" de los escaladores con menos de 3 años de experiencia

### 6.b) Dificultad media y maxima de los ascensos NO "Top Rope" de los escaladores con entre 10 y 30 años de experiencia

### 7. Los 10 riscos españoles (o zonas) con mas ascensos por orden decreciente de numero de ascensos

### 8.a) Los 10 sectores españoles con mayor nivel de difcultad media de ascensos ordenadas por orden decreciente de dificultad y por numero de ascensos decreciente

### 8.b) Las 10 sectores españoles con menor nivel de dificultad medio de ascensos ordenadas por orden creciente de dificultad y por numero de ascensos decreciente